# mini cluster of ESP32 - test

ref: [Celery Canvas](http://docs.celeryproject.org/en/latest/userguide/canvas.html)

The 'private' folder contains 'config_mqtt.py' with my private MQTT broker address and settings.  
For those who want to test this, please:  
- Upload all *.py found in this [repo.](https://github.com/Wei1234c/MQTT_bridged_workers) to ESP32's '/' folder.
- Copy all *.py found in this [repo.](https://github.com/Wei1234c/MQTT_bridged_workers) to a '...external/mqtt_network' folder in you local machine.
- UPload all *.py found in this repo. to ESP32's '/' folder.
- Config your copy of 'config_mqtt.py' in 'config' folder.
- Comment out the line with '... 'external', 'private' ...'.
- make sure the line with '... 'external', 'mqtt_network' ...' can refer to those *.py file you copy from this [repo.](https://github.com/Wei1234c/MQTT_bridged_workers).

### Imports

In [1]:
import os
import sys
import time

sys.path.append(os.path.abspath(os.path.join('..', '..', '..', 'external', 'private')))
sys.path.append(os.path.abspath(os.path.join('..', '..', '..', 'external', 'amd64')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'broccoli', 'client')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'broccoli', 'config')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'broccoli', 'node')))
sys.path.append(os.path.abspath(os.path.join('..', '..', '..', 'external', 'mqtt_network')))

import client
from collections import OrderedDict

from canvas import *
import tasks

My name is Wei-Legion


### Start client
啟動 client 物件。  

在本機上有一個 Broker 物件，負責：
- 管理本機上的 task queue
- 接受使用者發出的運算要求，並將之排入本機上的 task queue
- 通知遠端的 workers 協助處理 tasks
- 將工作發送給 workers 做處理
- 收集 workers 傳回的運算結果
- 將運算結果整合之後，傳回給使用者

而我們透過 client 物件來與 Broker 物件溝通

In [2]:
the_client = client.Client()
the_client.start()

while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')


Sending 281 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2018-04-10 17:51:35.848191 1'), ('kwargs', {'name': 'Wei-Legion'}), ('message_id', '2018-04-10 17:51:35.848191 1'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


[Connected: ('123.240.210.68', 1883)]
[Listen to messages]
Node not ready yet.


### Reset workers
如果需要確保 workers 的狀態都一致，或者需要重新 depoly Python module files 到 workers 上面去，可以發送指令給遠端的 workers，要求做 reboot 回到最初的狀態。

In [3]:
the_client.reset_workers()
time.sleep(15)  # wait until workers ready.


Sending 236 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:51:39.258540 10'), ('message_id', '2018-04-10 17:51:39.258540 10'), ('message_type', 'exec'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion'), ('to_exec', 'import machine;machine.reset()')])


Data received: 236 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:51:39.258540 10'), ('message_id', '2018-04-10 17:51:39.258540 10'), ('message_type', 'exec'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion'), ('to_exec', 'import machine;machine.reset()')])

No module named 'machine'

Data received: 257 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '6440'), ('kwargs', {'name': 'ESP32_b4e62d890c49'}), ('message_id', '6440'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Data received: 257 bytes
Message:
OrderedDict([('comman

### Upload tasks module file
Reboot 之後的 workers 只有最基本的功能，如果我們需要 workers 執行額外的功能 (functions)，則需要先將定義這些 functions 的 module 檔案上傳給 workers 並且要求它們 import。  

例如： 我們上傳給每一個 worker 一個`tasks.py`的檔案，其中定義幾個 functions:
```
from canvas import Task

@Task
def add(x, y, op=None):
    return op(x, y) if op else x + y

@Task
def xsum(x):
    return sum(x)

@Task
def mul(x, y, op=None):
    return op(x, y) if op else x * y

@Task
def mapper(word):
    return (word, 1) if len(word) > 3 else None

```

Workers 收到這個`tasks.py`檔案之後，會做`import tasks`的動作，所以就可以呼叫 tasks.add() 這個 function。

In [4]:
tasks_file = os.path.join('..', '..', 'codes', 'broccoli', 'client', 'tasks.py')
the_client.sync_file(tasks_file, load_as_tasks = True)


Sending 550 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:51:54.303603 23'), ('kwargs', {'filename': 'tasks.py', 'file': 'from canvas import Task\n\n@Task\ndef add(x, y, op=None):\n    return op(x, y) if op else x + y\n\n@Task\ndef xsum(x):\n    return sum(x)\n\n@Task\ndef mul(x, y, op=None):\n    return op(x, y) if op else x * y\n\n@Task\ndef mapper(word):\n    return (word, 1) if len(word) > 3 else None\n', 'load_as_tasks': True}), ('message_id', '2018-04-10 17:51:54.303603 23'), ('message_type', 'file'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])



<font color='blue'>
## DEMOs

### Chains
Celery [Chains](http://docs.celeryproject.org/en/latest/userguide/canvas.html#chains) 的主要作用是把多個運算**串聯**起來，前一個運算的結果是下一個運算的參數，這樣就可以組成一個完整的運算過程，例如下例中用`chain`組成一個 ((4+4) * 8) * 10  = 640 的計算過程
```
>>> # (4 + 4) * 8 * 10
>>> res = chain(add.s(4, 4), mul.s(8), mul.s(10))
proj.tasks.add(4, 4) | proj.tasks.mul(8) | proj.tasks.mul(10)

>>> res = chain(add.s(4, 4), mul.s(8), mul.s(10))()
>>> res.get()
640
```

我們可以在 ESP32 cluster 上面也做同樣的事情：

In [5]:
ch = chain(tasks.add.s(4, 4), tasks.mul.s(8), tasks.mul.s(10))
ch.get()


Sending 257 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:51:54.343706 27'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:51:54.343706 27'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 550 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:51:54.303603 23'), ('kwargs', {'filename': 'tasks.py', 'file': 'from canvas import Task\n\n@Task\ndef add(x, y, op=None):\n    return op(x, y) if op else x + y\n\n@Task\ndef xsum(x):\n    return sum(x)\n\n@Task\ndef mul(x, y, op=None):\n    return op(x, y) if op else x * y\n\n@Task\ndef mapper(word):\n    return (word, 1) if len(word) > 3 else None\n', 'load_as_tasks': True}), ('message_id', '2018-04-10 17:51:54.303603 23'), ('message_type', 'file'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 257 bytes
Message:
OrderedDict([('correlation_id', '20


Data received: 522 bytes
Message:
OrderedDict([('correlation_id', '16641'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 64, 'function': 'tasks.mul', 'args': [8, 8], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:51:56.058888 184', 'message_type': 'result', 'task_id': '2018-04-10 17:51:56.058888 184', 'correlation_id': '2018-04-10 17:51:56.058888 184', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '16641'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Sending 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:51:57.690963 351'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:51:57.690963 351'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:51:57.690963

640

### Groups
Celery [Groups](http://docs.celeryproject.org/en/latest/userguide/canvas.html#groups) 的主要作用是把多個運算**併聯**起來，把很多同質性的運算同時發送給許多遠端的 workers 協助處理，再收集 workers 傳回來的結果彙整成為一個結果集，例如下例中用`group`同時計算 (2+2) 和 (4+4)，結果是 [4, 8]
```
>>> from celery import group
>>> from proj.tasks import add

>>> group(add.s(2, 2), add.s(4, 4))
(proj.tasks.add(2, 2), proj.tasks.add(4, 4))  

>>> g = group(add.s(2, 2), add.s(4, 4))
>>> res = g()
>>> res.get()
[4, 8]
```
我們可以在 ESP32 cluster 上面也做同樣的事情：

In [6]:
gp = group([tasks.add.s(2, 2), tasks.add.s(4, 4)])
gp.get()


Sending 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:51:58.688300 463'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:51:58.688300 463'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:51:58.688300 463'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:51:58.688300 463'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '18136'), ('function', 'dequeue_task'), ('message_id', '18136'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '18136'), ('message_i

[4, 8]

我們可以用 iterators:
```
>>> group(add.s(i, i) for i in xrange(10))()
```

In [7]:
gp = group([tasks.add.s(i, i) for i in range(10)])
gp.get()


Sending 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:00.108408 577'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:00.108408 577'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:00.108408 577'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:00.108408 577'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '19499'), ('function', 'dequeue_task'), ('message_id', '19499'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d891439'), ('sender', 'ESP32_b4e62d891439')])


Processed result:
OrderedDict([('correlation_id', '19499'), ('message_i


Data received: 521 bytes
Message:
OrderedDict([('correlation_id', '21001'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 4, 'function': 'tasks.add', 'args': [2, 2], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:00.149517 581', 'message_type': 'result', 'task_id': '2018-04-10 17:52:00.149517 581', 'correlation_id': '2018-04-10 17:52:00.149517 581', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '21001'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '21104'), ('function', 'dequeue_task'), ('message_id', '21104'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '21104'), ('message_id', '2018-04-10 17:52:02.318353 829'), ('message_type', 



Sending 487 bytes
Message:
OrderedDict([('correlation_id', '22521'), ('message_id', '2018-04-10 17:52:03.754960 991'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d8904d1'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:52:00.149517 588', 'function': 'tasks.add', 'args': (9, 9), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:52:00.149517 588', 'task_id': '2018-04-10 17:52:00.149517 588'}, 0)), ('sender', 'Wei-Legion')])


Data received: 522 bytes
Message:
OrderedDict([('correlation_id', '22074'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 10, 'function': 'tasks.add', 'args': [5, 5], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:00.149517 584', 'message_type': 'result', 'task_id': '2018-04-10 17:52:00.149517 584', 'correlation_id': '2018-04-10 17:52:00.149517 584', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '22074'

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

### Chords
Celery [Chords](http://docs.celeryproject.org/en/latest/userguide/canvas.html#chords) 的主要作用是由兩段運算所組成的，第一段是一個`Groups`運算，其運算的結果會傳給第二段中的運算，作為其運算所需的參數。  

其作用可以用以下的例子來說明，`header`運算的結果會傳給`callback`做進一步的處理：
```
>>> callback = tsum.s()
>>> header = [add.s(i, i) for i in range(10)]
>>> result = chord(header)(callback)
>>> result.get()
90
```
我們可以在 ESP32 cluster 上面也做同樣的事情：

In [8]:
callback = tasks.xsum.s()
header = [tasks.add.s(i, i) for i in range(10)]
async_result = chord(header)(callback)
async_result.get()


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:04.693427 1100'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:04.693427 1100'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:04.693427 1100'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:04.693427 1100'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '24031'), ('function', 'dequeue_task'), ('message_id', '24031'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Processed result:
OrderedDict([('correlation_id', '24031'), ('messa


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '25190'), ('function', 'dequeue_task'), ('message_id', '25190'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d891439'), ('sender', 'ESP32_b4e62d891439')])


Processed result:
OrderedDict([('correlation_id', '25190'), ('message_id', '2018-04-10 17:52:06.347014 1292'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d891439'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:52:04.717464 1107', 'function': 'tasks.add', 'args': (5, 5), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:52:04.717464 1107', 'task_id': '2018-04-10 17:52:04.717464 1107'}, 4)), ('sender', 'Wei-Legion')])


Sending 491 bytes
Message:
OrderedDict([('correlation_id', '25190'), ('message_id', '2018-04-10 17:52:06.347014 1292'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d8


Sending 491 bytes
Message:
OrderedDict([('correlation_id', '26923'), ('message_id', '2018-04-10 17:52:08.179377 1496'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d8904d1'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:52:04.717464 1111', 'function': 'tasks.add', 'args': (9, 9), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:52:04.717464 1111', 'task_id': '2018-04-10 17:52:04.717464 1111'}, 0)), ('sender', 'Wei-Legion')])


Data received: 525 bytes
Message:
OrderedDict([('correlation_id', '26963'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 14, 'function': 'tasks.add', 'args': [7, 7], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:04.717464 1109', 'message_type': 'result', 'task_id': '2018-04-10 17:52:04.717464 1109', 'correlation_id': '2018-04-10 17:52:04.717464 1109', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '

90

上述的運算可以直接寫成：
```
chord(add.s(i, i) for i in xrange(10))(tsum.s()).get()
```
我們可以在 ESP32 cluster 上面也做同樣的事情：

In [9]:
async_result = chord([tasks.add.s(i, i) for i in range(10)])(tasks.xsum.s())
async_result.get()


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:11.563084 1903'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:11.563084 1903'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:11.563084 1903'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:11.563084 1903'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '30960'), ('function', 'dequeue_task'), ('message_id', '30960'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '30960'), ('messa

Message:
OrderedDict([('correlation_id', '31939'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 4, 'function': 'tasks.add', 'args': [2, 2], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:11.586189 1906', 'message_type': 'result', 'task_id': '2018-04-10 17:52:11.586189 1906', 'correlation_id': '2018-04-10 17:52:11.586189 1906', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '31939'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '32045'), ('function', 'dequeue_task'), ('message_id', '32045'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Processed result:
OrderedDict([('correlation_id', '32045'), ('message_id', '2018-04-10 17:52:13.769863 2155'), ('message_type', 'result'), ('receiver'


Data received: 525 bytes
Message:
OrderedDict([('correlation_id', '34128'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 14, 'function': 'tasks.add', 'args': [7, 7], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:11.587148 1911', 'message_type': 'result', 'task_id': '2018-04-10 17:52:11.587148 1911', 'correlation_id': '2018-04-10 17:52:11.587148 1911', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '34128'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d891439'), ('sender', 'ESP32_b4e62d891439')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '34232'), ('function', 'dequeue_task'), ('message_id', '34232'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d891439'), ('sender', 'ESP32_b4e62d891439')])


Processed result:
OrderedDict([('correlation_id', '34232'), ('message_id', '2018-04-10 17:52:15.477851 2337'), ('message_ty

90

### Map & Starmap
Celery [Map & Starmap](http://docs.celeryproject.org/en/latest/userguide/canvas.html#map-starmap) 的主要作用和 Python 中的`map`指令一樣，會對一個 list 中的每個 element 做指定的運算，例如下例會分別對`range(10)`,`range(100)`做`sum`運算：
```
>>> ~xsum.map([range(10), range(100)])
[45, 4950]
```
我們可以在 ESP32 cluster 上面也做同樣的事情，但是須先使用`list()`對`range`物件做展開：

In [10]:
gp = tasks.xsum.map([list(range(10)), list(range(100))])
gp.get()


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:17.206222 2522'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:17.206222 2522'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:17.206222 2522'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:17.206222 2522'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '36641'), ('function', 'dequeue_task'), ('message_id', '36641'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '36641'), ('messa

[45, 4950]

`starmap`的作用和`map`指令一樣，會對一個 list 中的每個 element 做指定的運算，只是會先做 star展開，將一個`list`展開成為 positional arguments：
```
>>> ~add.starmap(zip(range(10), range(10)))
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
```
我們可以在 ESP32 cluster 上面也做同樣的事情，但是須先使用`list()`對`zip`物件做展開：

In [11]:
gp = tasks.add.starmap(list(zip(range(10), range(10))))
gp.get()


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:18.807863 2698'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:18.807863 2698'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:18.807863 2698'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:18.807863 2698'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '38256'), ('function', 'dequeue_task'), ('message_id', '38256'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d891439'), ('sender', 'ESP32_b4e62d891439')])


Processed result:
OrderedDict([('correlation_id', '38256'), ('messa


Data received: 524 bytes
Message:
OrderedDict([('correlation_id', '39476'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 4, 'function': 'tasks.add', 'args': [2, 2], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:18.828918 2702', 'message_type': 'result', 'task_id': '2018-04-10 17:52:18.828918 2702', 'correlation_id': '2018-04-10 17:52:18.828918 2702', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '39476'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '39582'), ('function', 'dequeue_task'), ('message_id', '39582'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '39582'), ('message_id', '2018-04-10 17:52:21.757517 3046'), ('message_typ

Message:
OrderedDict([('correlation_id', '41791'), ('message_id', '2018-04-10 17:52:23.476241 3242'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d891439'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:52:18.828918 2709', 'function': 'tasks.add', 'args': (9, 9), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:52:18.828918 2709', 'task_id': '2018-04-10 17:52:18.828918 2709'}, 0)), ('sender', 'Wei-Legion')])


Data received: 525 bytes
Message:
OrderedDict([('correlation_id', '41982'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 14, 'function': 'tasks.add', 'args': [7, 7], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:18.828918 2707', 'message_type': 'result', 'task_id': '2018-04-10 17:52:18.828918 2707', 'correlation_id': '2018-04-10 17:52:18.828918 2707', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '41982'), ('message_

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

### Chunks
Celery [Chunks](http://docs.celeryproject.org/en/latest/userguide/canvas.html#chunks) 的主要作用是把一大串的資料切成指定的份數，分發給遠端的 workers 協處處理，例如：
```
>>> res = add.chunks(zip(range(100), range(100)), 10)()
>>> res.get()
[[0, 2, 4, 6, 8, 10, 12, 14, 16, 18],
 [20, 22, 24, 26, 28, 30, 32, 34, 36, 38],
 [40, 42, 44, 46, 48, 50, 52, 54, 56, 58],
 [60, 62, 64, 66, 68, 70, 72, 74, 76, 78],
 [80, 82, 84, 86, 88, 90, 92, 94, 96, 98],
 [100, 102, 104, 106, 108, 110, 112, 114, 116, 118],
 [120, 122, 124, 126, 128, 130, 132, 134, 136, 138],
 [140, 142, 144, 146, 148, 150, 152, 154, 156, 158],
 [160, 162, 164, 166, 168, 170, 172, 174, 176, 178],
 [180, 182, 184, 186, 188, 190, 192, 194, 196, 198]]
```
我們可以在 ESP32 cluster 上面也做同樣的事情，但是須先使用`list()`對`zip`物件做展開：

In [12]:
ck = tasks.add.chunks(list(zip(range(100), range(100))), 10)
async_result = ck()
async_result.get()


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:24.280980 3334'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:24.280980 3334'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:52:24.280980 3334'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:52:24.280980 3334'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '43622'), ('function', 'dequeue_task'), ('message_id', '43622'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Processed result:
OrderedDict([('correlation_id', '43622'), ('messa

Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '44739'), ('function', 'dequeue_task'), ('message_id', '44739'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d891439'), ('sender', 'ESP32_b4e62d891439')])


Processed result:
OrderedDict([('correlation_id', '44739'), ('message_id', '2018-04-10 17:52:26.004025 3619'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d891439'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:52:24.307051 3341', 'function': 'tasks.add', 'args': (5, 5), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:52:24.307051 3341', 'task_id': '2018-04-10 17:52:24.307051 3341'}, 94)), ('sender', 'Wei-Legion')])


Sending 492 bytes
Message:
OrderedDict([('correlation_id', '44739'), ('message_id', '2018-04-10 17:52:26.004025 3619'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d8



Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '46623'), ('function', 'dequeue_task'), ('message_id', '46623'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '46623'), ('message_id', '2018-04-10 17:52:28.175158 3859'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d890c49'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:52:24.307051 3346', 'function': 'tasks.add', 'args': (10, 10), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:52:24.307051 3346', 'task_id': '2018-04-10 17:52:24.307051 3346'}, 89)), ('sender', 'Wei-Legion')])


Sending 494 bytes
Message:
OrderedDict([('correlation_id', '46623'), ('message_id', '2018-04-10 17:52:28.175158 3859'), ('message_type', 'result'), ('receiver', 'ESP32_b4e

Message:
OrderedDict([('correlation_id', '48411'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 24, 'function': 'tasks.add', 'args': [12, 12], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:24.307051 3348', 'message_type': 'result', 'task_id': '2018-04-10 17:52:24.307051 3348', 'correlation_id': '2018-04-10 17:52:24.307051 3348', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '48411'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '48517'), ('function', 'dequeue_task'), ('message_id', '48517'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Processed result:
OrderedDict([('correlation_id', '48517'), ('message_id', '2018-04-10 17:52:29.952473 4052'), ('message_type', 'result'), ('receiv



Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '49843'), ('function', 'dequeue_task'), ('message_id', '49843'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '49843'), ('message_id', '2018-04-10 17:52:31.500241 4224'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d890c49'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:52:24.307051 3355', 'function': 'tasks.add', 'args': (19, 19), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:52:24.307051 3355', 'task_id': '2018-04-10 17:52:24.307051 3355'}, 80)), ('sender', 'Wei-Legion')])


Sending 494 bytes
Message:
OrderedDict([('correlation_id', '49843'), ('message_id', '2018-04-10 17:52:31.500241 4224'), ('message_type', 'result'), ('receiver', 'ESP32_b4e

...

Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '77865'), ('function', 'dequeue_task'), ('message_id', '77865'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '77865'), ('message_id', '2018-04-10 17:52:58.975350 7291'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d890c49'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:52:24.307051 3421', 'function': 'tasks.add', 'args': (85, 85), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:52:24.307051 3421', 'task_id': '2018-04-10 17:52:24.307051 3421'}, 14)), ('sender', 'Wei-Legion')])


Sending 494 bytes
Message:
OrderedDict([('correlation_id', '77865'), ('message_id', '2018-04-10 17:52:58.975350 7291'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62


Data received: 528 bytes
Message:
OrderedDict([('correlation_id', '79420'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 174, 'function': 'tasks.add', 'args': [87, 87], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:24.308053 3423', 'message_type': 'result', 'task_id': '2018-04-10 17:52:24.308053 3423', 'correlation_id': '2018-04-10 17:52:24.308053 3423', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '79420'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '79525'), ('function', 'dequeue_task'), ('message_id', '79525'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Processed result:
OrderedDict([('correlation_id', '79525'), ('message_id', '2018-04-10 17:53:00.754909 7487'), ('message


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '81115'), ('function', 'dequeue_task'), ('message_id', '81115'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d891439'), ('sender', 'ESP32_b4e62d891439')])


Processed result:
OrderedDict([('correlation_id', '81115'), ('message_id', '2018-04-10 17:53:02.691996 7706'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d891439'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:52:24.308053 3431', 'function': 'tasks.add', 'args': (95, 95), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:52:24.308053 3431', 'task_id': '2018-04-10 17:52:24.308053 3431'}, 4)), ('sender', 'Wei-Legion')])


Sending 493 bytes
Message:
OrderedDict([('correlation_id', '81115'), ('message_id', '2018-04-10 17:53:02.691996 7706'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62


Data received: 528 bytes
Message:
OrderedDict([('correlation_id', '83141'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 194, 'function': 'tasks.add', 'args': [97, 97], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:24.308053 3433', 'message_type': 'result', 'task_id': '2018-04-10 17:52:24.308053 3433', 'correlation_id': '2018-04-10 17:52:24.308053 3433', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '83141'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Data received: 528 bytes
Message:
OrderedDict([('correlation_id', '83415'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 196, 'function': 'tasks.add', 'args': [98, 98], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:52:24.308053 3434', 'message_type': 'result', 'task_id': '2018-04-10 17:52:24.308053 3434', 'correlation_id': '2018-04-10 17:52:24.308053 3434', 'need_result': 

[[0, 2, 4, 6, 8, 10, 12, 14, 16, 18],
 [20, 22, 24, 26, 28, 30, 32, 34, 36, 38],
 [40, 42, 44, 46, 48, 50, 52, 54, 56, 58],
 [60, 62, 64, 66, 68, 70, 72, 74, 76, 78],
 [80, 82, 84, 86, 88, 90, 92, 94, 96, 98],
 [100, 102, 104, 106, 108, 110, 112, 114, 116, 118],
 [120, 122, 124, 126, 128, 130, 132, 134, 136, 138],
 [140, 142, 144, 146, 148, 150, 152, 154, 156, 158],
 [160, 162, 164, 166, 168, 170, 172, 174, 176, 178],
 [180, 182, 184, 186, 188, 190, 192, 194, 196, 198]]

### Word Count
最後我們以 Hadoop 領域中的 "Hello World" 範例 "Word Count" 來測試。  

我們會把一個文字檔的內容拆解成 words 並將每個 word 發送給 workers 處理，workers 要做的主要是一個`mapper`處理：
```
def mapper(word):
    return (word, 1) if len(word) > 3 else None
```
worker 會將處理的結果傳回來，client 這邊會有一個`reduce`function 將結果彙整。  

我們可以在 ESP32 cluster 上面這樣做：

In [13]:
import word_count

text_file = os.path.join('..', '..', 'codes', 'broccoli', 'client', 'test.txt')
words_count, counts = word_count.count_words(text_file)
print('********** result:\nwords count: {}\n\n{}\n**********'.format(words_count, counts[:10]))


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:53:05.331304 8000'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:53:05.331304 8000'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 17:53:05.331304 8000'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Wei-Legion'}), ('message_id', '2018-04-10 17:53:05.331304 8000'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '84844'), ('function', 'dequeue_task'), ('message_id', '84844'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '84844'), ('messa


Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '85921'), ('function', 'dequeue_task'), ('message_id', '85921'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d891439'), ('sender', 'ESP32_b4e62d891439')])


Processed result:
OrderedDict([('correlation_id', '85921'), ('message_id', '2018-04-10 17:53:07.217639 8286'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d891439'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:53:05.344338 8006', 'function': 'tasks.mapper', 'args': ('Fyler',), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:53:05.344338 8006', 'task_id': '2018-04-10 17:53:05.344338 8006'}, 88)), ('sender', 'Wei-Legion')])


Data received: 530 bytes
Message:
OrderedDict([('correlation_id', '86207'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': None, 'function': 'tasks.


Sending 496 bytes
Message:
OrderedDict([('correlation_id', '87166'), ('message_id', '2018-04-10 17:53:08.562309 8433'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d891439'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:53:05.344338 8010', 'function': 'tasks.mapper', 'args': ('and',), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:53:05.344338 8010', 'task_id': '2018-04-10 17:53:05.344338 8010'}, 84)), ('sender', 'Wei-Legion')])


Data received: 531 bytes
Message:
OrderedDict([('correlation_id', '87743'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': None, 'function': 'tasks.mapper', 'args': ['The'], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:53:05.344338 8008', 'message_type': 'result', 'task_id': '2018-04-10 17:53:05.344338 8008', 'correlation_id': '2018-04-10 17:53:05.344338 8008', 'need_result': True, 'sender': 'Wei-Legion'}}), ('me

Message:
OrderedDict([('correlation_id', '89148'), ('function', 'dequeue_task'), ('message_id', '89148'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '89148'), ('message_id', '2018-04-10 17:53:10.450990 8637'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d890c49'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:53:05.344338 8015', 'function': 'tasks.mapper', 'args': ('with',), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:53:05.344338 8015', 'task_id': '2018-04-10 17:53:05.344338 8015'}, 79)), ('sender', 'Wei-Legion')])


Data received: 539 bytes
Sending 497 bytes
Message:
OrderedDict([('correlation_id', '89148'), ('message_id', '2018-04-10 17:53:10.450990 8637'), ('message_type', 'result'), ('receiver', 'ESP32_b

Data received: 219 bytes
Message:
OrderedDict([('correlation_id', '90618'), ('function', 'dequeue_task'), ('message_id', '90618'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d8904d1'), ('sender', 'ESP32_b4e62d8904d1')])


Processed result:
OrderedDict([('correlation_id', '90618'), ('message_id', '2018-04-10 17:53:12.031719 8807'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d8904d1'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:53:05.344338 8020', 'function': 'tasks.mapper', 'args': ('the',), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:53:05.344338 8020', 'task_id': '2018-04-10 17:53:05.344338 8020'}, 74)), ('sender', 'Wei-Legion')])


Sending 496 bytes
Message:
OrderedDict([('correlation_id', '90618'), ('message_id', '2018-04-10 17:53:12.031719 8807'), ('message_type', 'result'), ('receiver', 'ESP32_b4

...



Processed result:
OrderedDict([('correlation_id', '115575'), ('message_id', '2018-04-10 17:53:36.930329 11538'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d890c49'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:53:05.345340 8079', 'function': 'tasks.mapper', 'args': ('in',), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:53:05.345340 8079', 'task_id': '2018-04-10 17:53:05.345340 8079'}, 15)), ('sender', 'Wei-Legion')])


Sending 497 bytes
Message:
OrderedDict([('correlation_id', '115575'), ('message_id', '2018-04-10 17:53:36.930329 11538'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d890c49'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:53:05.345340 8079', 'function': 'tasks.mapper', 'args': ('in',), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17



Sending 499 bytes
Message:
OrderedDict([('correlation_id', '116764'), ('message_id', '2018-04-10 17:53:38.598614 11725'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d891439'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:53:05.345340 8083', 'function': 'tasks.mapper', 'args': ('good',), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:53:05.345340 8083', 'task_id': '2018-04-10 17:53:05.345340 8083'}, 11)), ('sender', 'Wei-Legion')])


Data received: 551 bytes
Message:
OrderedDict([('correlation_id', '117396'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': ['pasture"', 1], 'function': 'tasks.mapper', 'args': ['pasture"'], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:53:05.345340 8081', 'message_type': 'result', 'task_id': '2018-04-10 17:53:05.345340 8081', 'correlation_id': '2018-04-10 17:53:05.345340 8081', 'need_result': True, 'sender': 


Data received: 547 bytes
Message:
OrderedDict([('correlation_id', '119218'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': ['grossly', 1], 'function': 'tasks.mapper', 'args': ['grossly'], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:53:05.345340 8085', 'message_type': 'result', 'task_id': '2018-04-10 17:53:05.345340 8085', 'correlation_id': '2018-04-10 17:53:05.345340 8085', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '119218'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Data received: 221 bytes
Message:
OrderedDict([('correlation_id', '119325'), ('function', 'dequeue_task'), ('message_id', '119325'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])


Processed result:
OrderedDict([('correlation_id', '119325'), ('message_id', '2018-04-10 17:53:40.97


Data received: 221 bytes
Message:
OrderedDict([('correlation_id', '121079'), ('function', 'dequeue_task'), ('message_id', '121079'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d891439'), ('sender', 'ESP32_b4e62d891439')])


Processed result:
OrderedDict([('correlation_id', '121079'), ('message_id', '2018-04-10 17:53:42.848624 12202'), ('message_type', 'result'), ('receiver', 'ESP32_b4e62d891439'), ('reply_to', 'Wei-Legion'), ('result', ({'sender': 'Wei-Legion', 'message_type': 'function', 'message_id': '2018-04-10 17:53:05.345340 8092', 'function': 'tasks.mapper', 'args': ('yet',), 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 17:53:05.345340 8092', 'task_id': '2018-04-10 17:53:05.345340 8092'}, 2)), ('sender', 'Wei-Legion')])


Sending 497 bytes
Message:
OrderedDict([('correlation_id', '121079'), ('message_id', '2018-04-10 17:53:42.848624 12202'), ('message_type', 'result'), ('receiver', 'ES


Data received: 547 bytes
Message:
OrderedDict([('correlation_id', '124548'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': ['grass"', 1], 'function': 'tasks.mapper', 'args': ['grass"'], 'reply_to': 'Wei-Legion', 'message_id': '2018-04-10 17:53:05.345340 8094', 'message_type': 'result', 'task_id': '2018-04-10 17:53:05.345340 8094', 'correlation_id': '2018-04-10 17:53:05.345340 8094', 'need_result': True, 'sender': 'Wei-Legion'}}), ('message_id', '124548'), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_b4e62d890c49'), ('sender', 'ESP32_b4e62d890c49')])

********** result:
words count: 94

[('Lamb', 5), ('grossly', 3), ('Wolf', 2), ('year', 1), ('with', 1), ('voice', 1), ('violent', 1), ('tone', 1), ('thus', 1), ('then', 1)]
**********


In [14]:
# Stopping
the_client.stop()
the_client = None
print('\n[________________ Demo stopped ________________]\n')

[Closed: ('123.240.210.68', 1883)]

[________________ Demo stopped ________________]

